In [2]:
import folium
import json
from branca.colormap import linear
import geopandas
import ipywidgets as widgets
out = widgets.Output()

dataFile = "data/choropleth_data.geojson"
metaDataFile = "data/choropleth_metadata.json"
vectorTilesPBF= "http://openapi-dev.aurin.org.au/public/gwc/service/wmts?REQUEST=GetTile&SERVICE=WMTS&VERSION=1.0.0&LAYER=aurin:SA1_2016_AUST&STYLE=&TILEMATRIX=EPSG:900913:{z}&TILEMATRIXSET=EPSG:900913&FORMAT=application/x-protobuf;type=mapbox-vector&TILECOL={x}&TILEROW={y}"



#loading the local data
myData=geopandas.read_file(dataFile)

#load the linkingKey from metadata
metaData = json.load(open(metaDataFile))
keyField = metaData["key"]

numericFields = {}
#extract the numeric fields
for att in metaData["selectedAttributes"]:
    if(att['type'] == 'Short'):
        numericFields[att['description']] = att['name']


select = widgets.Select(
    options=numericFields,
    description='Select Field:',
    disabled=False
)

#Generate and display a map
def onFieldSelect(b):
    with out:
        field = select.value
        #generate a scalled color map
        colormap = linear.YlGn_09.scale(
            myData[field].min(),
            myData[field].max())

        #generate a key value dictionary with key as linking field and value as color to be applied
        linkData = {}
        for index, row in myData.iterrows():
            linkData[row[keyField]] =  colormap(row['negative_nil_income_tot'])
        #initialize a map
        m= folium.Map(tiles=None, location=[-37, 140],zoom_start=8)
        m.add_vector_grid_choropleth(vectorTilesPBF,
                                   name="SA1_2016_AUST",
                                   data=json.dumps(linkData),
                                   field=keyField.upper())
        folium.LayerControl().add_to(m)
        colormap.caption = select.label
        colormap.add_to(m)
        display(m)


button = widgets.Button(description="Show Map")
button.on_click(onFieldSelect)
controllComponent = widgets.VBox([select,button,out])
display(controllComponent)


    
